<h1 align="center"><font color="00308F" size=110>Ordinary and Partial Diferential Equations</font></h1>

This notebook supplements the lesson 4 course notes on ODEs and PDEs.

Let's start by setting up our environment and importing what we need.  

In [ ]:
import numpy as np
import matplotlib
%matplotlib inline

# Reminder: Difference between ODEs and PDEs

-   ODEs are differential equations containing **a single independent variable** so all the derivatives are ordinary dervatives. For example, given a scalar function $x \mapsto f(x)$, the first and second derivatives are $f'$ or $f''$.
-   PDEs are differential equations containing **two or more independent variables**, so the derivatives are partial derivatives. For example, given a function of two variables $(x,y) \mapsto f(x,y)$, the partial derivatives with respect to $x$ and $y$ are $\frac{\partial f}{\partial x}$ and $\frac{\partial f}{\partial y}$.

# PDE example: Heat (Diffusion) equation

The heat equation is a second order PDE describing how temperature $T$ diffuses through a medium of thermal diffusivity $\alpha$:

\begin{align}
&\text{General:} \qquad &\frac{\partial T}{\partial t} = D \nabla^2 T  \\
&\text{2D:} \qquad &\frac{\partial T}{\partial t} = D \left( \frac{\partial^2 T}{\partial x^2}  + \frac{\partial^2 T}{\partial y^2} \right)
\end{align}

# ODE example: Heat (Diffusion) equation

Keep the full PDE in mind, but let's start by simplifying the diffusion equation to 1D and time-independent:

$$ -D \frac{\partial^2 T}{\partial x^2} = 0 $$


## Review: Solution strategy for 1D diffusion equation


### First central difference for first $f'(x)$

Our aim with these methods is to replace the differential operator with an approximation which averages over nearby points and by using a mesh of such points we derive a set of simultaneous equations to solve. The trick is to apply Taylor's expansions at points near $f(x)$, such as $f(x-h)$ and $f(x+h)$, and to combine these expansions to obtain an approximation for the desired derivative.

Consider the following two Taylor expansions of a function f(x) around x at a (small) distance, h (see e.g. [?1])

\begin{align}
f(x + h) = f(x) + h f'(x) + \frac{h^2}{2!} f''(x) + \frac{h^3}{3!} f'''(x) + \frac{h^4}{4!} f^{(4)}(x) + O(h^5), \\
f(x - h) = f(x) - h f'(x) + \frac{h^2}{2!} f''(x) - \frac{h^3}{3!} f'''(x) + \frac{h^4}{4!} f^{(4)}(x) + O(h^5).\\
\end{align}

If we substract these two equations we get

\begin{align}
f(x + h) - f(x - h) = 2h f'(x) + \frac{h^3}{3} f'''(x) + O(h^5).
\end{align}

Rearranging gives

\begin{align}
f'(x) &= \frac{f(x + h) - f(x - h)}{2h} - \frac{h^2}{3} f'''(x) + O(h^4), \\
f'(x) &= \frac{f(x + h) - f(x - h)}{2h} + O(h^2).
\end{align}

This is known as the *first central difference approximation* for the *first* derivative of $x \mapsto f(x)$. It is accurate to order 2. It was derived by using Taylor's expansion for $f$ at two points around $x$: $x+h$ and $x-h$. These points are symmetric with respect to $x$. It is also possible to use *staggered* finite difference approximations such as the *first forward difference* for the first derivative, that expresses $f'(x)$ in terms of $f(x)$, $f(x + h)$ and $f(x + 2h)$.


### First central difference for $f''(x)$

It is also possible to obtain finite difference approximations for higher order derivatives. For example, if we add the two Taylor expansions for $f(x+h)$ and $f(x-h)$ instead of subtracting them we get

\begin{align}
f(x + h) + f(x - h) = 2 f(x) + h^2 f''(x) + O(h^4).
\end{align}

Rearranging yields

\begin{align}
f''(x) = \frac{f(x-h) - 2 f(x) + f(x + h)}{h^2} + O(h^2).
\end{align}

This is known as the *first central difference approximation* for the *second* derivative of $f(x)$.  The detailed derivation is included in the lesson 3 course notes.

### General central finite difference for $f^{(n)}(x)$

It is possible to derive these for higher order derivatives to various order of accuracy. A general central difference approximation for $f^{(n)}(x)$ can be written in the form

\begin{equation}
f^{(n)}(x) = \sum_{i=-m}^m  c_i f(x + i h),
\end{equation}

where $(c_i)_{-m \leq i \leq m}$ define the [*coefficients*](https://en.wikipedia.org/wiki/Finite_difference_coefficient) of the finite difference. The points $(x+ih)_{-m\leq i \leq m}$ are called the *nodes* of the finite difference.

## Bringing it all together: Numerical PDE Solution for Diffusion Equation

Let's solve the 1-D time-dependent heat equation given by:

$$ \frac{\partial T}{\partial t} - D \frac{\partial^2 T}{\partial x^2} = 0$$

Using the central difference approximation of the first and second derivative from above, we get:

$$ \frac{T(t + \Delta t) - T(t)}{\Delta t} - D \frac{T(x + \Delta x)-2T(x)+T(x - \Delta x)}{\Delta x^2} = 0$$

Rearranging,

$$ T(t + \Delta t) - T(t) - D \frac{\Delta t}{\Delta x^2} \left(T(x + \Delta x)-2T(x)+T(x - \Delta x) \right) = 0$$

We can replace $D \frac{\Delta t}{\Delta x^2}$ with s, the diffusion number, to get:

$$ T(t + \Delta t) - T(t) - s \left(T(x + \Delta x)-2T(x)+T(x - \Delta x) \right) = 0$$

### FTCS explicit scheme and analytic solution

If we use $n$ to refer to indices in time and $j$ to refer to indices in space, the above equation can be written as

$$ T[n + 1, j] = T[n, j] + s \left(T[n, j+1]-2T[n, j]+T[n, j-1] \right) = 0$$

This is called a forward-in-time, centered-in-space (FTCS) scheme. Its ‘footprint’ looks like this:

In [ ]:
plt.figure(figsize=(6,3))
plt.plot([0,2],[0,0],'k')
plt.plot([1,1],[0,1],'k')
plt.plot([0,1,2,1],[0,0,0,1],'ko',markersize=10)
plt.text(1.1,0.1,'T[n,j]')
plt.text(0.1,0.1,'T[n,j-1]')
plt.text(1.1,1.1,'T[n+1,j]')
plt.text(2.1,0.1,'T[n,j+1]')
plt.xlabel('space')
plt.ylabel('time')
plt.axis('equal')
plt.yticks([0.0,1.0],[])
plt.xticks([0.0,1.0],[])
plt.title('FTCS explicit scheme',fontsize=12)
plt.axis([-0.5,2.5,-0.5,1.5]);

Let's assume a 5 cm pipe with one end at 100 degrees Celsius.  The relevant constants have been defined below for you.

In [ ]:
# Relevant constand and initial values
dt = 0.0005 # grid size for time (s)
dx = 0.0005 # grid size for space (m)
D = 1.27*10**(-4) # thermal diffusivity of gold (m2/s)
x_max = 0.05 # in m
t_max = 5 # total time in s
T0 = 100 # Temp in Celsius

Now, using a FTCS scheme, write a function that can solve the diffusion equation:

In [ ]:
# function to calculate velocity profiles based on a 
# finite difference approximation to the 1D diffusion 
# equation and the FTCS scheme:
def diffusion_FTCS(dt, dx, t_max, x_max, D, T0):
    
    #! Compute the diffusion number
    
    
    #! Create sample points in space and time
    
    
    #! Create a matrix to store the solution
    
    
    #! Set initial values of the matrix
    
    
    #! Implement FTCS scheme

    
# Determine the numerical FTCS solution
x, T, r, s = diffusion_FTCS(dt, dx, t_max, x_max, D, T0)

# plotting:
plt.figure(figsize=(7,5))
plot_times = np.arange(0.01,5.0,0.2)
for t in plot_times:
    plt.plot(x , T[int(t/dt),:], 'Gray', label='numerical')
    if t==0.2:
        plt.legend(fontsize=12)
plt.xlabel('Distance from end of pipe (m)',fontsize=12)
plt.ylabel('Temperature [C]]',fontsize=12)
plt.axis([0, x_max, 0, T0])
plt.title('Explicit numerical (FTCS scheme) solution');

For this function, we have an analytic solution as well given by:

$$ T = T_0 \left( \sum^{\infty}_{n=0} erfc(2n \eta_1 + \eta) - \sum^{\infty}_{n=0} erfc(2(n+1)\eta_1 + \eta) \right) $$

where

$\eta=\frac{x}{2\sqrt{D t}}$ <br\>
$\eta_1=\frac{h}{2\sqrt{D t}}$

In [ ]:
from scipy.special import erfc
from math import sqrt
 
# Function to calculate velocity profiles using the analytic solution:
def diffusion_analytic(t, h, T0, dx, D):
    
    #! Add Code to compute the analytic solution
    
 
# plotting:
plt.figure(figsize=(7,5))
plot_times = np.arange(0.01,5.0,0.5)
for t in plot_times:
    plt.plot(x , T[int(t/dt),:], 'Gray', label='numerical')
    T_analytic = diffusion_analytic(t, x_max, T0, dx, D)
    plt.plot(x, T_analytic,'ok',label='analytic',
        markersize=3)
    if t==0.2:
        plt.legend(fontsize=12)
plt.xlabel('Distance from end of pipe [m]',fontsize=12)
plt.ylabel('Temperature [C]',fontsize=12)
plt.axis([0, x_max, 0, T0])
plt.title('Comparison between explicit numerical \n solution and analytic solution');


<fig04-04.pdf>



The dots (analytic solution) overlap pretty well with the lines (numerical solution). However, this would not be the case if we changed the discretization so that the diffusion number was larger. Let’s look at the stability of the FTCS numerical scheme, by computing the solution with different diffusion numbers. It turns out that the diffusion number s has to be less than 0.5 for the FTCS scheme to remain stable.

In [ ]:
dt = 0.0005 # grid size for time (m)
dx = 0.0005 # grid size for space (s)
x, T, r, s = diffusion_FTCS(dt, dx, t_max, x_max, D, T0)
T_analytic = diffusion_analytic(0.5, x_max, T0, dx, D)
plt.figure(figsize=(7,5))
plt.plot(x, T_analytic-T[int(0.5/dt)],'--k',label='small s')
 
dx = 0.0010
dt = 0.00254
x, T, r, s = diffusion_FTCS(dt, dx, t_max, x_max, D, T0)
T_analytic = diffusion_analytic(0.5, x_max, T0, dx, D)
T_numeric = T[r/2-1,:]
 
plt.plot(x, T_analytic-T_numeric,'k',label='large s')
plt.xlabel('Distance from end of pipe [m]',fontsize=12)
plt.ylabel('Temperature difference [C]',fontsize=12)
plt.title('Difference between numerical and analytic \n solutions for different \'s\' values',fontsize=14)
plt.axis([0, x_max, -100, 100])
plt.legend();

## Example using SCIPY: Radioactive Decay

The number of atoms left in a radioactive sample is given by:

$$ \frac{dN}{dt} = -\lambda N(t) $$

This is a linear, first order ODE.  The analytic solution is given as

$$N(t) = N_0e^{-\lambda t} $$

Let's compare [SciPy's odeint solver](https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.integrate.odeint.html) to the analytic solution.

In [ ]:
#! Define the radioactive decay function


# Define a 10 second half-life
lam = np.log(2)/10.

# Start with 100 atoms
N0 = 100.

#! Create a solution grid in time


#! Use ODEINT to solve


#! Plot


Now let's calculate the analytic solution and compare the difference. 

In [ ]:
# Analytic Solution
def decay_analytic(N0, lam, t):
    return N0*np.exp(-lam*t)

# Calculate the exact solution and difference
N_analytic = decay_analytic(N0, lam, t)
N_diff = np.abs(np.reshape(N,len(N)) - N_analytic)

# Plot the differences
plt.semilogy(t, N_diff)
plt.ylabel("Error")
plt.xlabel("Time [s]")
plt.title("Error in numerical solution");